<a href="https://colab.research.google.com/github/monash-tp37-project/iteration_1_model/blob/master/bert_model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load and use pretrained BERT model for news classification

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
#Installing the Hugging Face PyTorch Interface for Bert
# !pip install pytorch-pretrained-bert pytorch-nlp
!pip install -q transformers

In [15]:
import os
import torch
import torch.nn.functional as F
from transformers import BertTokenizer, BertConfig
from transformers import AdamW, BertForSequenceClassification, get_linear_schedule_with_warmup

In [16]:
current_dir = os.getcwd()
current_dir

'/content'

In [17]:
#import torch
#from transformers import AutoTokenizer, AutoModelForSequenceClassification

# read in trained tokenizer and model
tokenizer = BertTokenizer.from_pretrained(f"{current_dir}/drive/MyDrive/colab_data/tokenizer_200k")
model = BertForSequenceClassification.from_pretrained(f"{current_dir}/drive/MyDrive/colab_data/model_200k_128")

In [18]:
sequences = ["Previously unreported surveillance video captured one such effort in August in the rural Colorado town of Kiowa. Footage obtained by Reuters through a public-records request shows Elbert County Clerk Dallas Schroeder, the county’s top election official, fiddling with cables and typing on his phone as he copied computer drives containing sensitive voting information."]

# tokenize text input
tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

In [19]:
with torch.no_grad():
  # make predictions
  output = model(**tokens)

  # using a softmax activation function get discrete predictions
  predictions = F.softmax(output.logits, dim=1)
  labels = int(torch.argmax(predictions, dim=1))

  # using a sigmoid function get continous predictions 
  label = torch.sigmoid(predictions).tolist()[0]
  reliable = round(label[0],2)
  unreliable = round(label[1],2)
  print(f'Your news has a {unreliable}% probability of being unreliable and a {reliable}% of being reliable.')

  if reliable < unreliable:

    print('Therefore, our model predicts your news is likely to be unreliable.')

  else:

    print('Therefore, our model predicts your news is likely to be reliable.')

Your news has a 0.5% probability of being unreliable and a 0.73% of being reliable.
Therefore, our model predicts your news is likely to be reliable.


In [36]:
!pip3 install keybert
!pip3 install keyphrase-vectorizers
from keybert import KeyBERT
import keyphrase_vectorizers
from keyphrase_vectorizers import KeyphraseCountVectorizer

     |████████████████████████████████| 1.5 MB 10.8 MB/s 
     |████████████████████████████████| 6.2 MB 68.6 MB/s 
     |████████████████████████████████| 280 kB 72.7 MB/s 
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
     |████████████████████████████████| 749 kB 54.5 MB/s 
     |████████████████████████████████| 10.1 MB 31.1 MB/s 
     |████████████████████████████████| 653 kB 47.1 MB/s 
     |████████████████████████████████| 181 kB 61.5 MB/s 
     |████████████████████████████████| 457 kB 58.5 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 58 kB 6.2 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Att

In [37]:
kw_model = KeyBERT()

In [39]:
keywords = kw_model.extract_keywords(docs=sequences, vectorizer=KeyphraseCountVectorizer())

/usr/local/lib/python3.7/dist-packages/keybert/_model.py:131: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  "Although extracting keywords for multiple documents is faster "
1it [00:00, 434.15it/s]


In [47]:
[x[0] for x in sorted(keywords[0], key=lambda x: x[1], reverse=True)][0:3]

['elbert county clerk dallas schroeder',
 'sensitive voting information',
 'county']